# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
data_file = os.path.join("..","WeatherPy","exports","CityList.csv")
work_df = pd.read_csv(data_file)
work_df = work_df.drop(['Unnamed: 0'], axis=1)
work_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Richards Bay,-28.78,32.04,291.82,71,74,2.73,ZA,1600185697
1,Ushuaia,-54.80,-68.30,276.15,74,75,12.90,AR,1600185219
2,Riyadh,24.69,46.72,312.15,12,75,2.10,SA,1600185136
3,Palauig,15.44,119.90,300.46,76,100,1.72,PH,1600185698
4,Praia,14.92,-23.51,301.32,71,64,3.18,CV,1600185698


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
import gmaps.datasets
gmaps.configure(g_key)

,latitude,longitude,magnitude
0,65.193300,-149.072500,1.70
1,38.791832,-122.780830,2.10
2,38.818001,-122.792168,0.48
3,33.601667,-116.727667,0.78
4,37.378334,-118.520836,3.64


In [9]:
locations = work_df[['Lat', 'Lng']]
weights = work_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
work_df_filtered = work_df[work_df["Max Temp"] < 300] #degrees are in Kelvin, need to go back and fix. 300 Kelvin is close to 80 degrees F
work_df_filtered = work_df_filtered[work_df_filtered["Max Temp"] > 294] #degrees are in Kelvin, need to go back and fix. 294 Kelvin is close to 70 degrees F
work_df_filtered = work_df_filtered[work_df_filtered["Wind Speed"]<= 10]
work_df_filtered = work_df_filtered[work_df_filtered["Humidity"]<= 60]
work_df_filtered.shape



(24, 9)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# geocoordinates for the base of our search
target_location = "29.714033,-95.406480" # coordinates

# distance, IN METERS, within which the place results must live
target_radius = 5000

# keyword term to be matched against all of Google's indexed content
#target_keyword = "Chinese"

# type of establishment that we'd like to use to filter our place results
target_type = "hotel"

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a dictionary to store all query parameters
params = {
    "location": target_location,
#    "keyword": target_keyword,
    "radius": target_radius,
#     "rankby": "distance",
    "type": target_type,
    "key": g_key
}

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [ ]:
marker_locations = [
    (-34.0, -59.166672),
    (-32.23333, -64.433327),
    (40.166672, 44.133331),
    (51.216671, 5.0833302),
    (51.333328, 4.25)
]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
